In [1]:
import cv2
import numpy as np
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

import time
from yolov4.tf import YOLOv4

yolo = YOLOv4()
yolo.classes = "coco.names"
yolo.make_model()
yolo.load_weights("../weights/yolov4.weights", weights_type="yolo")

In [2]:
def pre_process(img):
    resized_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_image = yolo.resize_image(resized_image)
    resized_image = resized_image / 255
    input_data = resized_image[np.newaxis, ...].astype(np.float32)
    
    return input_data
    

In [3]:
# original_image == Dim(height, width, channels)
original_image = cv2.imread("kite.jpg")
input_data = pre_process(original_image)

# Saving/Loading model

In [ ]:
# Saving model
model = yolo.model
save_path = ''
tf.saved_model.save(model, save_path)

In [4]:
# Loading model
save_path = ''
st = time.time()
imported = tf.saved_model.load(save_path)
print(time.time() - st)
tf_model = imported.signatures["serving_default"]

85.57219862937927


# Loaded prediction

In [124]:
def loaded_predict(processed_img):
    
    ## load model if not already loaded.
    
    batch_data = tf.constant(processed_img)
    
    st = time.time()
    candidates = tf_model(batch_data)
#     print(time.time() - st)
    
#     st = time.time()
    candidates = (candidates["output_1"].numpy(), candidates["output_2"].numpy(), candidates["output_3"].numpy())
#     print(time.time() - st)
    
    return candidates

In [118]:
candidates = loaded_predict(input_data)

0.09275197982788086


# Original way prediction

In [121]:
start_time = time.time()
candidates = yolo.model.predict(input_data)
print(time.time() - start_time)


0.11768507957458496


# Post-processing

In [ ]:
start_time = time.time()
_candidates = []
for candidate in candidates:
    batch_size = candidate.shape[0]
    grid_size = candidate.shape[1]
    _candidates.append(
        tf.reshape(
            candidate, shape=(1, grid_size * grid_size * 3, -1)
        )
    )
# candidates == Dim(batch, candidates, (bbox))
candidates = np.concatenate(_candidates, axis=1)

# pred_bboxes == Dim(candidates, (x, y, w, h, class_id, prob))
pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0], 0.3, 0.25)
pred_bboxes = yolo.fit_pred_bboxes_to_original(
    pred_bboxes, original_image.shape
)
exec_time = time.time() - start_time
print("time: {:.2f} ms".format(exec_time * 1000))

In [ ]:
result = yolo.draw_bboxes(original_image, pred_bboxes)
cv2.imwrite("result.jpg", result) 
from IPython.display import Image
Image("result.jpg")

In [122]:
def add_BBs(img, candidates):
    start_time = time.time()
    _candidates = []
    for candidate in candidates:
        batch_size = candidate.shape[0]
        grid_size = candidate.shape[1]
        _candidates.append(
            tf.reshape(
                candidate, shape=(1, grid_size * grid_size * 3, -1)
            )
        )
    # candidates == Dim(batch, candidates, (bbox))
    candidates = np.concatenate(_candidates, axis=1)

    # pred_bboxes == Dim(candidates, (x, y, w, h, class_id, prob))
    pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0], 0.3, 0.25)
    pred_bboxes = yolo.fit_pred_bboxes_to_original(
        pred_bboxes, img.shape
    )
    exec_time = time.time() - start_time
#     print("time: {:.2f} ms".format(exec_time * 1000))

    result = yolo.draw_bboxes(img, pred_bboxes)
    
    return result
    
    

# Video test 

In [126]:
cap = cv2.VideoCapture(0)

fps = 5
cap.set(cv2.CAP_PROP_POS_FRAMES, fps)

while True:
    ret, frame = cap.read()
    input_data = pre_process(frame)
    
    st = time.time()
    candidates = yolo.model.predict(input_data)
#     candidates = loaded_predict(input_data)
    st = time.time()
    frame = add_BBs(frame, candidates)
    
    delay = round(time.time() - st, 4)
    
    
    cv2.putText(frame, str(delay), (40,40), 1, 2, (0, 0, 255), 3)
    cv2.imshow("frame", frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
x = [1, 2, 3 ,4]

In [ ]:
sum(x) / len(x)